# 1. Simple Linear Regression 복습
 - 하나의 정보로부터 하나의 결론을 짓는 모델이다.
 - H(x) = Wx + b
 - ex) 공부 시간에 따른 시험 성적 예측
 

<br>
<hr>
<br>

# 2. Multivariate Linear Regression 이론
 - 복수의 정보로부터 하나의 결롯을 짓는 모델이다.
 - H(x) = w1x1 + w2x2 + w3x3 + b (입력 변수가 3개면 weight도 3개!)
 - ex) 첫번째 퀴즈 성적, 두번째 퀴즈 성적, 세번째 퀴즈 성적을 통해 final 시험을 예측한다.


<br>
<hr>
<br>

# 3. Naive Data Representation

### (1) 데이터 정의

In [1]:
# 데이터
import torch

x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]]) # 첫번째 퀴즈 성적
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]]) # 두번째 퀴즈 성적
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]]) # 세번째 퀴즈 성적
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]]) # final 시험

### (2) Hypothesis function 정의 - 기본
 - 아래와 같이 입력 변수의 수 만큼 weight를 곱했다.
 - 하.지.만. 입력 변수가 1000개라면..?

In [2]:
w1 = torch.zeros(1, requires_grad = True)
w2 = torch.zeros(1, requires_grad = True)
w3 = torch.zeros(1, requires_grad = True)
b = torch.zeros(1, requires_grad = True)

hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

### (3) Hypothesis function 정의 - matmul
 - matmul()로 한번에 정의가 가능하다. x의 길이가 바뀌어도 코드를 바꿀 필요가 없고, 속도도 더 빠르다.

In [3]:
W = torch.zeros(1, requires_grad = True)
x_train = torch.FloatTensor([[1], [2], [3]]) # 입력

hypothesis = x_train.matmul(W) + b

### (4) 그 외
 - Cost function : MSE로 Linear regression과 동일!
 - Gradient Descent : SGD로 Linear regression과 동일!

<br>
<hr>
<br>

# 4. Full Code with torch.optim
 - Simple linear regression과 달라지는 부분은 데이터, W 밖에 없다.

In [4]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True) # 3개의 독립변수!
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = torch.optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7967, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936096
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371071
Epoch    6/20 hypothesis: tensor([148.1035, 178.0143, 175.3980, 191.0042, 135.7812]) Cost: 29.758249
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8509, 193.6753, 137.6805]) Cost: 10.445267
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391237
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493121
Epoch   10/20 hypo

# 5. nn.Module 소개
 - 모델이 커질수록 W와 b를 일일히 치기 귀찮아 질 수 있다.

In [5]:
import torch.nn as nn

class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # nn.Linear에 입력 차원과 출력 차원을 설정한다.

    def forward(self, x):
        return self.linear(x) # forward 함수에 hypothesis 계산을 어떻게 할지 설정한다.

model = MultivariateLinearRegressionModel()
hypothesis = model(x_train)

# 6. F.mse_loss 소개
 - PyTorch에서는 다양한 cost(loss) function을 제공한다.
 - 원래 배웠던 것처럼 cost function을 직접 작성해도 되지만, 라이브러리를 이용하면 다른 loss function들과 쉽게 교체가 가능하다.
 - cost function 코드를 작성하면서 생길 수 있는 오류를 염려 안해도 된다.

In [6]:
import torch.nn.functional as F

cost = F.mse_loss(hypothesis, y_train) # MSE!

# 7. Full Code with torch.optim
 - nn.Module과 F.mse_loss를 적용한 코드!

In [7]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
# 모델 초기화
model = MultivariateLinearRegressionModel() # 변형됨!
# optimizer 설정
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train) # 변형됨!
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train) # 변형됨!
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 26242.500000
Epoch    1/20 Cost: 8226.089844
Epoch    2/20 Cost: 2578.901855
Epoch    3/20 Cost: 808.807861
Epoch    4/20 Cost: 253.977295
Epoch    5/20 Cost: 80.067360
Epoch    6/20 Cost: 25.555700
Epoch    7/20 Cost: 8.468952
Epoch    8/20 Cost: 3.113012
Epoch    9/20 Cost: 1.434006
Epoch   10/20 Cost: 0.907538
Epoch   11/20 Cost: 0.742365
Epoch   12/20 Cost: 0.690405
Epoch   13/20 Cost: 0.673938
Epoch   14/20 Cost: 0.668616
Epoch   15/20 Cost: 0.666758
Epoch   16/20 Cost: 0.666001
Epoch   17/20 Cost: 0.665595
Epoch   18/20 Cost: 0.665287
Epoch   19/20 Cost: 0.665014
Epoch   20/20 Cost: 0.664757
